In [37]:
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNet, HuberRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.exceptions import ConvergenceWarning
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import warnings
import joblib
import logging
import os

logging.disable(logging.CRITICAL)

# Suppress warnings for convergence issues
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Function to find optimal max_iter for a model
def find_optimal_iterations(model, X_train, y_train, initial_iter=5000, max_iter_limit=50000, step=5000):
    current_iter = initial_iter
    while current_iter <= max_iter_limit:
        model.set_params(max_iter=current_iter)
        try:
            model.fit(X_train, y_train)
            return model
        except ConvergenceWarning:
            current_iter += step
    return model

# Function to preprocess the data
def preprocess_data(df, numerical_features, categorical_features):
    interaction_terms = [
        'IELTS Score: Reading * Course Duration',
        'IELTS Score: Listening * IELTS Score: Reading',
        'IELTS Score: Writing * IELTS Score: Speaking'
    ]
    
    for term in interaction_terms:
        base, interaction = term.split('*')
        df[f'Interaction: {term}'] = df[base.strip()] * df[interaction.strip()]
    
    X_interactions = pd.concat([df[numerical_features], df[[f'Interaction: {term}' for term in interaction_terms]]], axis=1)
    X_interactions.columns = X_interactions.columns.astype(str)

    # Drop rows with NaN values across all features and target columns
    X_interactions = X_interactions.dropna()

    poly = PolynomialFeatures(degree=3, include_bias=False)
    poly_features = poly.fit_transform(X_interactions)

    joblib.dump(poly, 'poly.pkl')

    poly_df = pd.DataFrame(poly_features, columns=poly.get_feature_names_out(X_interactions.columns.astype(str)))

    df_poly_interactions = pd.concat([df[categorical_features].reset_index(drop=True), poly_df], axis=1)
    df_poly_interactions.columns = df_poly_interactions.columns.astype(str)

    preprocessor_pipeline = Pipeline(steps=[
        ('preprocessor', ColumnTransformer(
            transformers=[('cat', OneHotEncoder(drop='first'), categorical_features)],
            remainder='passthrough'
        )),
        ('scaler', StandardScaler(with_mean=False))
    ])

    X_poly_interactions = preprocessor_pipeline.fit_transform(df_poly_interactions)

    joblib.dump(preprocessor_pipeline, 'preprocessor_pipeline.pkl')

    return X_poly_interactions, df.index  # Return index to maintain alignment

# Function to train and evaluate models
def train_and_evaluate(df, targets, X_poly_interactions, original_index):
    best_models = {}
    results = {}

    for target in targets:
        df_target_dropped = df.dropna(subset=[target])
        X_aligned = X_poly_interactions[original_index.isin(df_target_dropped.index)]
        
        y = df_target_dropped[target]
        X_train, X_test, y_train, y_test = train_test_split(X_aligned, y, test_size=0.2, random_state=42)

        if target == 'Final Listening %' or target == 'Final speaking %':
            model = ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=42)
            model_name = 'ElasticNet'
        elif target == 'Final Reading %':
            model = SVR(kernel='poly', C=1, coef0=1, degree=2, gamma='auto')
            model_name = 'Support Vector Regressor (SVR)'
        elif target == 'Final Writing %':
            model = HuberRegressor(alpha=1, epsilon=1.35, max_iter=5000)
            model_name = 'Huber Regressor'

        model.fit(X_train, y_train)

        best_models[target] = model

        y_pred = model.predict(X_aligned)
        rmse = np.sqrt(mean_squared_error(y, y_pred))
        r2 = r2_score(y, y_pred)
        mae = mean_absolute_error(y, y_pred)
        results[target] = (model_name, rmse, r2, mae)

    joblib.dump(best_models, 'best_models_with_optimal_iterations_and_evaluation.pkl')

    return results

# Function to preprocess input for prediction
def preprocess_input(user_input):
    preprocessor_pipeline = joblib.load('preprocessor_pipeline.pkl')
    poly = joblib.load('poly.pkl')
    
    numerical_features = ['IELTS Total Score', 'IELTS Score: Listening', 'IELTS Score: Reading', 
                          'IELTS Score: Writing', 'IELTS Score: Speaking', 'Course Duration']
    interaction_terms = [
        'IELTS Score: Reading * Course Duration',
        'IELTS Score: Listening * IELTS Score: Reading',
        'IELTS Score: Writing * IELTS Score: Speaking'
    ]
    categorical_features = ['Gender', 'Nationality', 'Distance Learning', 'Future course']
    
    user_input_df = pd.DataFrame([user_input])

    for term in interaction_terms:
        base, interaction = term.split('*')
        user_input_df[f'Interaction: {term}'] = user_input_df[base.strip()] * user_input_df[interaction.strip()]
    
    user_numerical_input = user_input_df[numerical_features + [f'Interaction: {term}' for term in interaction_terms]]
    
    user_poly_features = poly.transform(user_numerical_input)

    user_poly_features_df = pd.DataFrame(user_poly_features, columns=poly.get_feature_names_out())

    cat_data = user_input_df[categorical_features]
    cat_data.columns = cat_data.columns.astype(str)

    input_final = preprocessor_pipeline.transform(pd.concat([cat_data, user_poly_features_df], axis=1))

    return input_final

# Function to clip predictions between 0 and 100
def clip_predictions(predictions):
    return np.clip(predictions, 0, 100)

# Function to make predictions and ensure they are within 0-100%
def make_predictions(input_final):
    best_models = joblib.load('best_models_with_optimal_iterations_and_evaluation.pkl')
    
    final_scores = {}
    for target, model in best_models.items():
        prediction = model.predict(input_final)
        final_scores[target] = clip_predictions(prediction[0])
    return final_scores

# Function to make conservative predictions by accounting for MAE
def make_conservative_predictions(final_scores, results):
    conservative_scores = {}
    for target, score in final_scores.items():
        conservative_scores[target] = max(0, score - results[target][3])
    return conservative_scores

def evaluate_against_course_requirements(df, final_scores, results, future_course, user_input, show_main_optimal=True, show_conservative=True, show_standard_recommendation=True):
    current_duration = user_input['Course Duration']
    initial_dl = user_input['Distance Learning']

    course_row = df[df['Future course'] == future_course].iloc[0]

    overall_required = course_row['Overall Required %']
    minimum_required = course_row['Minimum Required %']

    # Convert final_scores to floats
    numeric_final_scores = {k: float(v) for k, v in final_scores.items()}
    
    overall_percentage = np.mean(list(numeric_final_scores.values()))
    requirements_met = overall_percentage >= overall_required
    overall_result = "Pass" if requirements_met else "Fail"

    # Prepare data for the table
    table_data = {
        "Metric": [],
        "Score": [],
        "Required": [],
        "Result": [],
    }

    warnings_list = []
    
    for score_type, score_value in numeric_final_scores.items():
        mae = results[score_type][3]  # Extract the MAE for the specific model
        result = "Pass" if score_value >= minimum_required else "Fail"

        table_data["Metric"].append(score_type)
        table_data["Score"].append(f"{score_value:.2f}")
        table_data["Required"].append(f"{minimum_required:.2f}")
        table_data["Result"].append(result)

        # Add warnings if close to the minimum requirement
        if score_value >= minimum_required and (score_value - mae < minimum_required):
            warnings_list.append(f"Warning: {score_type}: {score_value:.2f} is close to the minimum required {minimum_required:.2f}. "
                                 f"There is a possibility of the student failing due to prediction uncertainty (MAE: {mae:.2f}).")

    # Add overall percentage to the table
    table_data["Metric"].append("Overall Percentage")
    table_data["Score"].append(f"{overall_percentage:.2f}")
    table_data["Required"].append(f"{overall_required:.2f}")
    table_data["Result"].append(overall_result)

    # Convert the table data to a DataFrame for nicer formatting
    results_df = pd.DataFrame(table_data)

    # Display the table with borders
    print("\nEvaluation Results (Based on Model Predictions):")
    print(results_df.to_markdown(index=False, tablefmt="grid"))
    
    # Display the overall requirements
    print(f"Requirements Met: {'Yes' if requirements_met else 'No'}")

    # Print warnings if any
    if warnings_list:
        print("\nWarnings:")
        for warning in warnings_list:
            print(warning)

    print("-" * 50)

    # Suggestions based on requirements not met
    suggestion = ""

    # Define available durations regardless of conditions
    available_durations = [6, 10, 15, 20, 31, 41]

    # Show Standard Course Duration Recommendation only if requirements are not met and show_standard_recommendation is True
    if not requirements_met and show_standard_recommendation:
        # List to store suggestions for both distance learning options
        main_suggestions = {"Yes": None, "No": None}

        for duration in available_durations:
            for dl_option in ["Yes", "No"]:
                if duration in [31, 41] and dl_option == 'Yes':
                    continue  # Skip invalid combinations

                test_user_input = user_input.copy()  # Use the original user_input passed to the function
                test_user_input['Course Duration'] = duration
                test_user_input['Distance Learning'] = dl_option

                input_final = preprocess_input(test_user_input)
                predicted_scores = make_predictions(input_final)

                overall_percentage = np.mean(list(predicted_scores.values()))

                # Check if the suggestion meets the overall and section requirements
                if overall_percentage >= overall_required and all(score >= minimum_required for score in predicted_scores.values()):
                    if not main_suggestions[dl_option]:
                        main_suggestions[dl_option] = duration

        if any(main_suggestions.values()):
            # Append suggestions to the string instead of printing immediately
            suggestion += f"Standard Course Duration Recommendation:\n"
            if main_suggestions["Yes"]:
                suggestion += f"- Distance Learning = Yes: {main_suggestions['Yes']} weeks\n"
            if main_suggestions["No"]:
                suggestion += f"- Distance Learning = No: {main_suggestions['No']} weeks\n"

    # Show Cautious Course Duration Recommendation if conservative or explicitly required
    if show_conservative or (not requirements_met and not any(main_suggestions.values())):
        conservative_scores = make_conservative_predictions(numeric_final_scores, results)
        conservative_overall_percentage = np.mean(list(conservative_scores.values()))
        conservative_requirements_met = conservative_overall_percentage >= overall_required

        if not conservative_requirements_met:
            conservative_suggestions = {"Yes": None, "No": None}

            for duration in available_durations:
                for dl_option in ["Yes", "No"]:
                    if duration in [31, 41] and dl_option == 'Yes':
                        continue  # Skip invalid combinations

                    test_user_input = user_input.copy()
                    test_user_input['Course Duration'] = duration
                    test_user_input['Distance Learning'] = dl_option

                    input_final = preprocess_input(test_user_input)
                    predicted_scores = make_predictions(input_final)

                    conservative_scores = {k: v - 0.5 * results[k][3] for k, v in predicted_scores.items()}
                    conservative_overall_percentage = np.mean(list(conservative_scores.values()))

                    if conservative_overall_percentage >= overall_required and all(score >= minimum_required for score in conservative_scores.values()):
                        if not conservative_suggestions[dl_option]:
                            conservative_suggestions[dl_option] = duration

            if any(conservative_suggestions.values()):
                # Append cautious recommendations to the suggestion string
                suggestion += f"\nCautious Course Duration Recommendation:\n"
                if conservative_suggestions["Yes"]:
                    suggestion += f"- Distance Learning = Yes: {conservative_suggestions['Yes']} weeks\n"
                if conservative_suggestions["No"]:
                    suggestion += f"- Distance Learning = No: {conservative_suggestions['No']} weeks\n"

            if not any(conservative_suggestions.values()):
                suggestion += "No alternative course duration and distance learning combination found that meets the requirements."

    # Print the final suggestion only once
    if suggestion:
        print(suggestion)

    return numeric_final_scores, overall_percentage, requirements_met, warnings_list, suggestion


def process_excel_file(file_path, results, df):
    input_df = pd.read_excel(file_path)

    # Drop rows with any missing values
    input_df = input_df.dropna()

    predictions = []
    recommendations = []
    warnings_summary = []

    skipped_records = 0

    for index, row in input_df.iterrows():
        user_input = row.to_dict()
        future_course = user_input['Future course']

        # Ensure future_course exists in the DataFrame
        if future_course not in df['Future course'].values:
            skipped_records += 1
            continue  # Skip the record if the course does not exist

        # Preprocess input
        input_final = preprocess_input(user_input)
        final_scores = make_predictions(input_final)

        # Evaluate results with cautious recommendations
        numeric_final_scores, overall_percentage, requirements_met, warnings_list, suggestion = evaluate_against_course_requirements(
            df, final_scores, results, future_course, user_input, 
            show_main_optimal=False,  # We don't want the standard recommendation
            show_conservative=True,  # We want the cautious course duration to be recommended
            show_standard_recommendation=False  # Ensure standard recommendation is not shown
        )

        # Store predictions with all original columns
        prediction_result = row.to_dict()  # Copy all original data
        prediction_result.update(numeric_final_scores)
        prediction_result['Overall Percentage'] = overall_percentage
        prediction_result['Requirements Met'] = requirements_met
        predictions.append(prediction_result)

        # Store recommendations if requirements not met
        if not requirements_met:
            recommendation_result = row.to_dict()
            recommendation_result.update({'Cautious Course Duration Recommendation': suggestion})
            recommendations.append(recommendation_result)

        # Store warnings with all original columns
        for warning in warnings_list:
            warning_result = row.to_dict()  # Copy all original data
            warning_result['Warning'] = warning
            warnings_summary.append(warning_result)

    # Convert lists to DataFrames
    predictions_df = pd.DataFrame(predictions)
    recommendations_df = pd.DataFrame(recommendations)
    warnings_df = pd.DataFrame(warnings_summary)

    # Save results to a new Excel file
    output_file = os.path.splitext(file_path)[0] + '_output.xlsx'
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        predictions_df.to_excel(writer, sheet_name='Predictions', index=False)
        recommendations_df.to_excel(writer, sheet_name='Recommendations', index=False)
        warnings_df.to_excel(writer, sheet_name='Warnings', index=False)

    # Print summary after processing all records
    print(f"Results have been saved to {output_file}")
    if skipped_records > 0:
        print(f"{skipped_records} records were skipped because the future course did not exist in the dataset.")
    print(f"Processed {len(predictions_df)} records successfully.")
    if not recommendations_df.empty:
        print(f"{len(recommendations_df)} records require cautious course duration recommendations.")
    if not warnings_df.empty:
        print(f"{len(warnings_df)} warnings were generated.")


def main():
    user_choice = input("Please choose an option:\n1. Cautious Course Duration Recommendation\n2. Full Evaluation Process\n3. Batch Process with Excel File\nEnter 1, 2, or 3: ").strip()

    if user_choice not in ['1', '2', '3']:
        print("Invalid choice. Please restart and choose either 1, 2, or 3.")
        return

    file_path = 'Book3.xlsx'  # Replace with your actual file path
    df = pd.read_excel(file_path)

    selected_columns = [
        'IELTS Total Score', 'IELTS Score: Listening', 'IELTS Score: Reading', 
        'IELTS Score: Writing', 'IELTS Score: Speaking', 
        'Gender', 'Nationality', 'Course Duration', 
        'Distance Learning', 'Final Listening %', 
        'Final Reading %', 'Final Writing %', 'Final speaking %',
        'Future course', 'Overall Required %', 'Minimum Required %'
    ]
    df = df[selected_columns].dropna()

    numerical_features = [
        'IELTS Total Score', 'IELTS Score: Listening', 'IELTS Score: Reading', 
        'IELTS Score: Writing', 'IELTS Score: Speaking', 'Course Duration'
    ]
    categorical_features = ['Gender', 'Nationality', 'Distance Learning', 'Future course']

    X_poly_interactions, original_index = preprocess_data(df, numerical_features, categorical_features)

    targets = ['Final Listening %', 'Final Reading %', 'Final Writing %', 'Final speaking %']

    results = train_and_evaluate(df, targets, X_poly_interactions, original_index)

    if user_choice == '1':
        # Cautious Course Duration Recommendation
        user_input = {
            'IELTS Total Score': float(input("Enter IELTS Total Score: ")),
            'IELTS Score: Listening': float(input("Enter IELTS Score: Listening: ")),
            'IELTS Score: Reading': float(input("Enter IELTS Score: Reading: ")),
            'IELTS Score: Writing': float(input("Enter IELTS Score: Writing: ")),
            'IELTS Score: Speaking': float(input("Enter IELTS Score: Speaking: ")),
            'Gender': input("Enter Gender (Male/Female): "),
            'Nationality': input("Enter Nationality: "),
            'Future course': input("Enter Future Course: ")
        }

        # Default values for Course Duration and Distance Learning
        user_input['Course Duration'] = 6
        user_input['Distance Learning'] = 'Yes'

        future_course = user_input['Future course']

        if future_course not in df['Future course'].values:
            print(f"Error: The future course '{future_course}' does not exist in the dataset.")
            return

        input_final = preprocess_input(user_input)
        final_scores = make_predictions(input_final)

        # Evaluate with conservative approach
        evaluate_against_course_requirements(df, final_scores, results, future_course, user_input, show_main_optimal=False, show_standard_recommendation=False)

    elif user_choice == '2':
        # Full Evaluation Process
        show_stats = input("Do you want to see model performance statistics? (yes/no): ").strip().lower()

        if show_stats == 'yes':
            # Output the performance results for the entire dataset
            print("\nPerformance on the entire dataset:")
            for target, (model_name, rmse, r2, mae) in results.items():
                print(f"Performance for {target} using {model_name}:")
                print(f"RMSE: {rmse:.4f}")
                print(f"R²: {r2:.4f}")
                print(f"MAE: {mae:.4f}\n")

        user_input = gather_user_input()

        future_course = user_input['Future course']

        if future_course not in df['Future course'].values:
            print(f"Error: The future course '{future_course}' does not exist in the dataset.")
            return

        input_final = preprocess_input(user_input)
        final_scores = make_predictions(input_final)

        # Evaluate normally and suggest standard course duration if needed
        evaluate_against_course_requirements(df, final_scores, results, future_course, user_input, show_main_optimal=True, show_standard_recommendation=True)

    elif user_choice == '3':
        # Handle batch processing
        excel_file_path = input("Enter the path to the Excel file with student records: ").strip()

        if not os.path.exists(excel_file_path):
            print(f"Error: The file '{excel_file_path}' does not exist.")
            return

        process_excel_file(excel_file_path, results, df)

if __name__ == "__main__":
    main()
